In [2]:
from log_analysis_functions import *

# Log Analysis

The purpose of this notebook is to analyse the logs that have been produced from tests with errors. In most of the cases, a single VM (the one that tends to contain malicious subscribers receiving data from a DOS attack) tends to not restart and we need to understand if the CPU usage or the network usage built up to a point of no return so to say.

We start off by analysing the following tests where VM 4 didn't restart when it was supposed to.

- `3_45_dos_unicast_3_512_kilobytes`
- `3_51_dos_unicast_3_1024_kilobytes`
- `3_51_dos_unicast_3_1024_kilobytes_rerun`
- `3_54_dos_multicast_3_1024_kilobytes`

## 3_45_dos_unicast_3_512_kilobytes

`_metadata` file show the following:

```
Test Start: [28/03/22]: 19:46:30
Restart VM 10.200.51.21: [28/03/22]: 20:01:48
Restart VM 10.200.51.23: [28/03/22]: 20:01:50
Restart VM 10.200.51.22: [28/03/22]: 20:06:22
Restart VM 10.200.51.24: [28/03/22]: 20:06:28
Restart VM 10.200.51.22: [28/03/22]: 20:22:51
Restart VM 10.200.51.21: [28/03/22]: 20:22:57
Restart VM 10.200.51.23: [28/03/22]: 20:23:00
```
We need to analyse what happened between 20:06:28 to 20:23:00 on VM 10.200.51.23 A.K.A VM 4 but the logs don't exist for some reason.

## 3_51_dos_unicast_3_1024_kilobytes / 3_51_dos_unicast_3_1024_kilobytes_rerun

`_metadata` file shows the following:

```
Test Start: [04/04/22]: 14:23:25
Restart VM 10.200.51.22: [04/04/22]: 14:39:40
Restart VM 10.200.51.21: [04/04/22]: 14:39:42
Restart VM 10.200.51.23: [04/04/22]: 14:39:48
```

We need to find out what happened following up to and aroudn 14:39 on VM 4 (10.200.51.24).

Turns out the logs for VM 4 doesn't exist either...

## 3_54_dos_multicast_3_1024_kilobytes

`_metadata` file shows the following:

```
Test Start: [04/04/22]: 16:47:20
Restart VM 10.200.51.21: [04/04/22]: 17:03:41
Restart VM 10.200.51.23: [04/04/22]: 17:03:45
Restart VM 10.200.51.22: [04/04/22]: 17:09:21
```

We are interested in finding out what the logs tell us what happened around 17:05 on VM 4 (10.200.51.24).

In [1]:
def parse_log(type, file):
    # console.print(Markdown("# " + file), style="")
    if "cpu" in type:
        with open(file, 'r') as f:
            file_contents = f.readlines()
        title_line = [line for line in file_contents[0:5] if '%usr' in line][0]

        titles = ["Time"] + [item.replace("%", "").replace("\n", "") for item in title_line.split(" ") if item != "" and not any(char.isdigit() for char in item)]
        
        times = []
        usr = []
        nice = []
        sys = []
        iowait = []
        idle = []

        title_line_index = file_contents.index(title_line)

        measurements = file_contents[title_line_index + 1:]

        for measurement in measurements:
            values = [item for item in measurement.split(" ") if item != ""]
            if len(values) > 3:
                try:
                    console.print(file_contents[0:10], style="green")
                    time_value = [int(value) for value in values[0].split(":")]
                except Exception as e:
                    print(file_contents[0:10])
                times.append(datetime.time(time_value[0], time_value[1], time_value[2]))
                usr.append(values[2])
                sys.append(values[4])
                iowait.append(values[5])
                idle.append(values[len(values) - 1])
        
        parsed_measurements = {
            "times": times,
            "usr": usr,
            "sys": sys,
            "iowait": iowait,
            "idle": idle
        }

        return parsed_measurements

    elif "network" in type:
        None
    else:
        console.print("Invalid type given to function: " + str(type), style="bold red")

# Get log files
all_logs = [file for file in get_files('data/set_3/3_54_dos_multicast_3_1024_kilobytes') if '.log' in file]
cpu_logs = [file for file in all_logs if 'cpu_usage' in file]
network_logs = [file for file in all_logs if 'network_usage' in file]

fig, ax = plt.subplots(nrows=len(cpu_logs), ncols=1, figsize=(20, 40))

for file in cpu_logs:
    data = parse_log('cpu', file)
    index = cpu_logs.index(file)
    ax[index].set_title(file)

NameError: name 'get_files' is not defined